# Helmet Detection - YOLOv8 Pipeline

This notebook handles the entire pipeline:
1. **Setup**: Imports and directory validation.
2. **Dataset Preparation**: Converts VOC XML annotations to YOLO format, splits data into Train/Validation, and organizes folders.
3. **Configuration**: Generates the `data.yaml` file required by YOLO.
4. **Training**: Fine-tunes a YOLOv8 model on the dataset.

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import random
import yaml
from tqdm.notebook import tqdm
from ultralytics import YOLO

# Ensure reproducibility
random.seed(42)

In [ ]:
# --- CONFIGURATION ---
# Value from your previous setup
PROJECT_ROOT = r"C:\Users\jsnal\OneDrive - Bina Nusantara\Semester 3\Deep Learning\AOL Project"
SOURCE_DATASET = os.path.join(PROJECT_ROOT, "dataset")
SOURCE_IMAGES = os.path.join(SOURCE_DATASET, "images")
SOURCE_ANNOTATIONS = os.path.join(SOURCE_DATASET, "annotations")

# Destination for prepared YOLO dataset
DEST_DATASET = os.path.join(PROJECT_ROOT, "datasets", "helmet_dataset")
TRAIN_IMAGES_DIR = os.path.join(DEST_DATASET, "train", "images")
TRAIN_LABELS_DIR = os.path.join(DEST_DATASET, "train", "labels")
VAL_IMAGES_DIR = os.path.join(DEST_DATASET, "val", "images")
VAL_LABELS_DIR = os.path.join(DEST_DATASET, "val", "labels")

In [ ]:
def get_all_classes(annotations_dir):
    """Scans all XML files to find unique class names."""
    classes = set()
    xml_files = [f for f in os.listdir(annotations_dir) if f.endswith('.xml')]
    print(f"Scanning {len(xml_files)} XML files for classes...")
    
    for xml_file in tqdm(xml_files, desc="Scanning Classes"):
        try:
            tree = ET.parse(os.path.join(annotations_dir, xml_file))
            root = tree.getroot()
            for obj in root.findall('object'):
                name = obj.find('name').text
                classes.add(name)
        except Exception as e:
            print(f"Error reading {xml_file}: {e}")
    
    return sorted(list(classes))

def convert_box(size, box):
    """Converts min/max coordinates to YOLO norm x,y,w,h."""
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    return (x * dw, y * dh, w * dw, h * dh)

In [ ]:
# --- PREPARE DATASET ---

# 1. Find Classes
classes = get_all_classes(SOURCE_ANNOTATIONS)
print(f"Detected Classes: {classes}")

# 2. Create Directories (Clean start)
for d in [TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_IMAGES_DIR, VAL_LABELS_DIR]:
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

# 3. Split Data
xml_files = [f for f in os.listdir(SOURCE_ANNOTATIONS) if f.endswith('.xml')]
random.shuffle(xml_files)

split_ratio = 0.8
split_index = int(len(xml_files) * split_ratio)
train_files = xml_files[:split_index]
val_files = xml_files[split_index:]

print(f"Total Files: {len(xml_files)}")
print(f"Training Set: {len(train_files)} images")
print(f"Validation Set: {len(val_files)} images")

def process_batch(files, img_dest, label_dest, batch_name="Processing"):
    for xml_file in tqdm(files, desc=batch_name):
        try:
            xml_path = os.path.join(SOURCE_ANNOTATIONS, xml_file)
            tree = ET.parse(xml_path)
            root = tree.getroot()
            
            # Image filename
            filename = root.find('filename').text
            src_img_path = os.path.join(SOURCE_IMAGES, filename)
            
            # Check if image exists, handle extension mismatches if needed
            if not os.path.exists(src_img_path):
                # Try common extensions
                basename = os.path.splitext(xml_file)[0]
                for ext in ['.jpg', '.png', '.jpeg', '.JPG']:
                    if os.path.exists(os.path.join(SOURCE_IMAGES, basename + ext)):
                        src_img_path = os.path.join(SOURCE_IMAGES, basename + ext)
                        filename = basename + ext
                        break
                
            if not os.path.exists(src_img_path):
                print(f"Skipping {xml_file}, image not found.")
                continue

            # Image Size
            size = root.find('size')
            w = int(size.find('width').text)
            h = int(size.find('height').text)

            # Convert Labels
            label_str = []
            for obj in root.findall('object'):
                cls_name = obj.find('name').text
                if cls_name in classes:
                    cls_id = classes.index(cls_name)
                    xmlbox = obj.find('bndbox')
                    b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
                         float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
                    bb = convert_box((w, h), b)
                    label_str.append(f"{cls_id} {bb[0]:.6f} {bb[1]:.6f} {bb[2]:.6f} {bb[3]:.6f}")

            # Save Labels to txt
            txt_name = os.path.splitext(filename)[0] + ".txt"
            with open(os.path.join(label_dest, txt_name), 'w') as f:
                f.write('\n'.join(label_str))
            
            # Copy Image
            shutil.copy(src_img_path, os.path.join(img_dest, filename))
            
        except Exception as e:
            print(f"Error processing {xml_file}: {e}")

process_batch(train_files, TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR, "Training Data")
process_batch(val_files, VAL_IMAGES_DIR, VAL_LABELS_DIR, "Validation Data")
print("Dataset prepared successfully!")

In [ ]:
# --- GENERATE DATA.YAML ---
yaml_content = {
    'path': DEST_DATASET,
    'train': 'train/images',
    'val': 'val/images',
    'names': {i: name for i, name in enumerate(classes)}
}

yaml_path = os.path.join(PROJECT_ROOT, "data.yaml")
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f, sort_keys=False)

print(f"Configuration saved to: {yaml_path}")
print(yaml_content)

In [ ]:
# --- START TRAINING ---
# Load a pretrained YOLOv8 nano model
model = YOLO('yolov8n.pt') 

# Train the model
# - data: path to our data.yaml
# - epochs: number of training epochs (start with 50-100 for decent results)
# - imgsz: image size (640 is standard)
# - device: 0 for GPU, 'cpu' for CPU
results = model.train(
    data=yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    device='0',  # Use '0' if you have one GPU, or 'cpu'
    project='helmet_detection_project',
    name='yolov8n_run1'
)